Make sure the cell below evaluates before continuing:

In [ ]:
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import scipy as sp
from scipy.interpolate import LSQUnivariateSpline
from PIL import Image
from IPython.display import Image as Im
from IPython.display import display

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift

# Cross Validation: The Right and Wrong Way (Lecture 6)

As we mentioned in Lecture 6 it is not uncommon to see scientists use Cross Validation the wrong way. See [this blog post](http://followthedata.wordpress.com/2013/10/30/the-importance-of-proper-cross-validation-and-experimental-design/) about a study which claimed to have found a genetic signature for autism, which supposedly could be used for a screening test for autism risk at birth or in infancy. The authors had first performed feature selection on the whole data set and then  divided the data set into a training set and a validation set to assess the performance.

So let us model this to see what happens.

## The scenario

You have 20 data points, each of which has 1,000,000 attributes. Each observation also has an associated $y$ value, and you are interested in whether a linear combination of a few attributes can be used to predict $y$. That is, you are looking for a model

$$
y_i \sim \sum_j \beta_j x_{ij}
$$

where most of the 1 million $\beta_j$ values are 0.

## The problem

Since there are so many more attributes than data points, the chance that a few attributes correlate with $y$ by pure coincidence is fairly high. 

If you think about it,  cross-validation helps you detect over-fitting, but you're perhaps fuzzy on the details.

## The wrong way to cross-validate

* Determine a few attributes of $X$ that correlate well with $Y$
* Use cross-validation to measure how well a linear fit to these attributes predicts $y$

Let's make the dataset, and compute the $y$'s with a "hidden" model that we are trying to recover:

In [ ]:
def hidden_model(x):
    #y is a linear combination of columns 5 and 10...
    result = x[:, 5] + x[:, 10]
    #... with a little noise
    result += np.random.normal(0, .005, result.shape)
    return result
    
def make_x(num_obs):
    return np.random.uniform(0, 3, (num_obs, 10 ** 6))

x = make_x(20)
y = hidden_model(x)

print(x.shape)

Now try to find the 2 attributes in $X$ that best correlate with $Y$. Recall from Lecture 7 we could do feature selection, and in fact the `sklearn.feature_selection.SelectKBest` will give us the `k` best features with the lowest $p$-value:

In [ ]:
selector = SelectKBest(f_regression, k=2).fit(x, y)
best_features = np.where(selector.get_support())[0]
print(best_features)

We know we are already in trouble --- we've selected 2 columns which correlate with $Y$ by chance, but neither of which are columns 5 or 10 (the only 2 columns that *actually* have anything to do with $Y$). We can look at the correlations between these columns and $Y$, and confirm they are pretty good (again, just a coincidence):

In [ ]:
for b in best_features:
    plt.plot(x[:, b], y, 'o')
    plt.title("Column %i" % b)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

A linear regression on the full data with these two features looks good. The "score" here is the $R^2$ score --- scores close to 1 imply a good fit.

In [ ]:
xt = x[:, best_features]
clf = LinearRegression().fit(xt, y)
print("Score is ", clf.score(xt, y))

In [ ]:
yp = clf.predict(xt)
plt.plot(yp, y, 'o')
plt.plot(y, y, 'r-')
plt.xlabel("Predicted")
plt.ylabel("Observed")

So now the model fits so well, we are worried about overfitting, but let's use cross-validation to detect this via `sklearn.cross_validation.cross_val_score`. Let's look at the average $R^2$ score, when performing 5-fold cross validation. It's not as good, but still not bad...

In [ ]:
cross_val_score(clf, xt, y, cv=5).mean()

And even if we make a plot of the predicted and actual data at each cross-validation iteration, the model seems to predict the "independent" data pretty well...

In [ ]:
for train, test in KFold(len(y), 10):
    xtrain, xtest, ytrain, ytest = xt[train], xt[test], y[train], y[test]

    clf.fit(xtrain, ytrain)
    yp = clf.predict(xtest)
    
    plt.plot(yp, ytest, 'o')
    plt.plot(ytest, ytest, 'r-')
    

plt.xlabel("Predicted")
plt.ylabel("Observed")

**But** --- what if we generated some more data?

In [ ]:
x2 = make_x(100)
y2 = hidden_model(x2)
x2 = x2[:, best_features]

y2p = clf.predict(x2)

plt.plot(y2p, y2, 'o')
plt.plot(y2, y2, 'r-')
plt.xlabel("Predicted")
plt.ylabel("Observed")

Yikes --- there is no correlation at all! Cross-validation did **not** detect the overfitting, because we used the entire data to select "good" features before hand!

## The right way to Cross-Validate

To prevent overfitting, we can't let *any* information about the full dataset leak into cross-validation. Thus, we must re-select good features in each cross-validation iteration:

In [ ]:
scores = []

for train, test in KFold(len(y), n_folds=5):
    xtrain, xtest, ytrain, ytest = x[train], x[test], y[train], y[test]
    
    b = SelectKBest(f_regression, k=2)
    b.fit(xtrain, ytrain)
    xtrain = xtrain[:, b.get_support()]
    xtest = xtest[:, b.get_support()]
    
    clf.fit(xtrain, ytrain)    
    scores.append(clf.score(xtest, ytest))

    yp = clf.predict(xtest)
    plt.plot(yp, ytest, 'o')
    plt.plot(ytest, ytest, 'r-')
    
plt.xlabel("Predicted")
plt.ylabel("Observed")

print("CV Score is ", np.mean(scores))

Now cross-validation properly detects overfitting, by reporting a low average $R^2$ score and a plot that looks like noise. Of course, it doesn't help us actually discover the fact that columns 5 and 10 determine $Y$ (this task is probably hopeless without more data) --- it just lets us know when our fitting approach isn't generalizing to new data.

---

# Feature Selection (Lecture 7)

Here we apply the best subset selection approach to the `Hitters` data. We wish to predict a baseball player’s `Salary` on the basis of various statistics associated with performance in the previous year. We drop any missing rows in place.

In [ ]:
hitters_df = pd.read_csv("../data/Hitters.csv", index_col ='Name')
hitters_df.dropna(inplace=True)
hitters_df.head()

Because this is a regression problem, we first need to convert the non-numeric input variables to factors. We will use the `pd.factorize` here to encode the categorical features as dummy variables:

In [ ]:
hitters_df["League"] = pd.factorize(hitters_df["League"])[0]
hitters_df["Division"] = pd.factorize(hitters_df["Division"])[0]
hitters_df["NewLeague"] = pd.factorize(hitters_df["NewLeague"])[0]
hitters_df.head()

We construct a baseline regressor with all features:

In [ ]:
collist = [col for col in hitters_df.columns if col != "Salary"]
X = hitters_df[collist]
y = hitters_df["Salary"]
reg = LinearRegression()
reg.fit(X, y)
ypred = reg.predict(X)
base_mse = np.sqrt(mean_squared_error(ypred, y))
print("Baseline Training MSE is ", base_mse)

## Best Subset Regression

Use `SelectKBest` to plot the mean square error for each `k` in `n_features = range(1, len(collist))`. We use the `get_support()` function to get the index of the features selected:

In [ ]:
mses = []
n_features = range(1, len(collist))
for k in n_features:
    # compute MSE for different values of k (top features)
    selector = SelectKBest(f_regression, k=k)
    selector.fit(X, y)
    selected = selector.get_support()
    feats = [col for (col,sel) in zip(collist, selected) if sel]
    reg = LinearRegression()
    X_r = hitters_df[feats]
    reg.fit(X_r, y)
    ypred = reg.predict(X_r)
    mses.append(np.sqrt(mean_squared_error(ypred, y)))

plt.plot(n_features, mses)
plt.xlabel("k")
plt.ylabel("RMSE")
plt.show()

## Model Selection by Cross-Validation (Your Turn)

The RMSE falls as the number of features increase --- this is expected because we are computing the RMSE from the training set (overfitting). We will now use 10-fold cross validation on each model to calculate a cross-validation RMSE which will give us a better idea of the best feature size to use for the problem:

In [ ]:
cv_errors = []
kfold = KFold(len(hitters_df), n_folds=10)
n_features = range(1, len(collist))
for k in n_features:
    # build model with varying number of features
    selector = SelectKBest(f_regression, k=k)
    selector.fit(X, y)
    selected = selector.get_support()
    feats = [col for (col,sel) in zip(collist, selected) if sel]
    
    # and then use the train and test in kfold to calculate CV MSE:
    rmses = []
    for train, test in kfold:
        # each model is cross validated 10 times
        Xtrain, ytrain, Xtest, ytest = X[train], y[train], X[test], y[test]
        reg = LinearRegression()
        reg.fit(Xtrain, ytrain)
        ypred = reg.predict(Xtest)
        rmses.append(np.sqrt(mean_squared_error(ytest, ypred)))
        
    # CV MSE is the mean of the CV rmses:
    cv_errors.append(np.mean(rmses))
plt.plot(n_features, cv_errors)
plt.xlabel("k")
plt.ylabel("RMSE")

## Ridge Regression and the Lasso (Your Turn)

Recall that these methods improve the model by shrinking the coefficients. The difference was in the norm used as the second constraint. Ridge regression used $\lambda \sum_{j=1}^{p} \beta_{j}^{2}$ where $\lambda \ge 0$, so a L2 norm. Lasso used $ \lambda \sum_{j=1}^{p} |\beta_{j}|$ as the penalty and hence an L1 norm.

Here we use cross validation to compute the RMSE for a baseline model, a model regularized by Ridge regression and one regularized using Lasso. Note the regularisation parameter $\lambda$ is called `alpha` here.

In [ ]:
def cross_validate(X, y, nfolds, reg_name):
    rmses = []
    kfold = KFold(X.shape[0], n_folds=nfolds)
    for train, test in kfold:
        Xtrain, ytrain, Xtest, ytest = X[train], y[train], X[test], y[test]
        reg = None
        if reg_name == "ridge":
            reg = Ridge()
        elif reg_name == "lasso":
            reg = Lasso()
        else:
            reg = LinearRegression()
        reg.fit(Xtrain, ytrain)
        ypred = reg.predict(Xtest)
        rmses.append(np.sqrt(mean_squared_error(ytest, ypred)))
    return np.mean(rmses)

rmse_baseline = cross_validate(X.values, y.values, 10, "baseline")
rmse_ridge = cross_validate(X.values, y.values, 10, "ridge")
rmse_lasso = cross_validate(X.values, y.values, 10, "lasso")
(rmse_baseline, rmse_ridge, rmse_lasso)

Find an optimum value of `alpha` for the Lasso regressor using cross-validation:

In [ ]:
cv_errors = []
alphas = [0.1 * alpha for alpha in range(1, 250, 20)]
kfold = KFold(X.shape[0], n_folds=10)
for alpha in alphas:
    rmses = []
    for train, test in kfold:
        Xtrain, ytrain, Xtest, ytest = X.values[train], y.values[train], X.values[test], y.values[test]
        reg = Lasso(alpha=alpha)
        reg.fit(Xtrain, ytrain)
        ypred = reg.predict(Xtest)
        rmses.append(np.sqrt(mean_squared_error(ytest, ypred)))
        
    cv_errors.append(np.mean(rmses))
plt.plot(alphas, cv_errors)
plt.xlabel("alpha")
plt.ylabel("RMSE")

We can also plot the Lasso coefficients as a function of the regularization:

In [ ]:
alphas = [0.1 * alpha for alpha in range(1, 250, 20)]
reg = Lasso()
coefs = []
for alpha in alphas:
    reg.set_params(alpha=alpha)
    reg.fit(X,y)
    coefs.append(reg.coef_)

plt.plot(alphas, coefs)
plt.xlabel('alpha')
plt.ylabel('weights')

---

# Dimension Reduction: PCA (Lecture 7)

Recall PCA from Lecture 7 where we project the data onto a lower-dimensional space and use that as our new features.  The principle behind the projections is captured below in this plot from stack overflow:

![](../data/pcavsfit.png)

We will use PCA to distinguish between cash and check at an ATM. We have some bolierplate code first to standardize the size of the images and then read all 87 of them from the data directory:

In [ ]:
#setup a standard image size; this will distort some images but will get everything into the same shape
STANDARD_SIZE = (322, 137)
def img_to_matrix(filename, verbose=False):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    if verbose==True:
        print "changing size from %s to %s" % (str(img.size), str(STANDARD_SIZE))
    img = img.resize(STANDARD_SIZE)
    img = list(img.getdata())
    img = map(list, img)
    img = np.array(img)
    return img

def flatten_image(img):
    """
    takes in an (m, n) numpy array and flattens it 
    into an array of shape (1, m * n)
    """
    s = img.shape[0] * img.shape[1]
    img_wide = img.reshape(1, s)
    return img_wide[0]

checks_dir = "../data/checks/"
dollars_dir = "../data/dollars/"
def images(img_dir):
    return [img_dir+f for f in os.listdir(img_dir)]
checks=images(checks_dir)
dollars=images(dollars_dir)
images=checks+dollars
labels = ["check" for i in range(len(checks))] + ["dollar" for i in range(len(dollars))]
len(labels), len(images)

Lets see what some of these images look like:

In [ ]:
for i in range(3):
    display(Im(checks[i]))
for i in range(3):
    display(Im(dollars[i]))

What features might you use to distinguish the cash notes from the checks? Here is an example of transforming an image into its R channel:

In [ ]:
i0=images[20]
display(Im(i0))
i0m=img_to_matrix(i0)
print i0m.shape
plt.imshow(i0m[:,1].reshape(137,322))

We can do this for every image, flattening each image into 3 channels of 44114 pixels, for a total of 132342 features per image!

In [ ]:
data = []
for image in images:
    img = img_to_matrix(image)
    img = flatten_image(img)
    data.append(img)

data = np.array(data)
data.shape

In [ ]:
y = np.where(np.array(labels)=="check", 1, 0)
y.shape

We now carryout a 20D PCA, which captures 73% of the variance:

In [ ]:
def do_pca(d,n):
    pca = PCA(n_components=n)
    X = pca.fit_transform(d)
    print pca.explained_variance_ratio_
    return X, pca

In [ ]:
X20, pca20=do_pca(data,20)

In [ ]:
np.sum(pca20.explained_variance_ratio_)

Just for kicks, because we can plot it, we'll do the 2D PCA:

In [ ]:
X2, pca2=do_pca(data,2)

In [ ]:
df = pd.DataFrame({"x": X2[:, 0], "y": X2[:, 1], "label":np.where(y==1, "check", "dollar")})
colors = ["red", "yellow"]
for label, color in zip(df['label'].unique(), colors):
    mask = df['label']==label
    plt.scatter(df[mask]['x'], df[mask]['y'], c=color, label=label)
plt.legend()

A quick visual shows that 2Dims may be enough to allow for linear separation of checks from dollars, with 42% of the variance accounted for. 

We provide some code below to reconstruct, from the principal components, the images corresponding to them:

In [ ]:
def normit(a):
    a=(a - a.min())/(a.max() -a.min())
    a=a*256
    return np.round(a)

def getRGB(o):
    size=322*137*3
    r=o[0:size:3]
    g=o[1:size:3]
    b=o[2:size:3]
    r=normit(r)
    g=normit(g)
    b=normit(b)
    return r,g,b

def getNC(pc, j):
    return getRGB(pc.components_[j])

def getMean(pc):
    m=pc.mean_
    return getRGB(m)

def display_from_RGB(r, g, b):
    rgbArray = np.zeros((137,322,3), 'uint8')
    rgbArray[..., 0] = r.reshape(137,322)
    rgbArray[..., 1] = g.reshape(137,322)
    rgbArray[..., 2] = b.reshape(137,322)
    img = Image.fromarray(rgbArray)
    plt.imshow(np.asarray(img))
    ax=plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

def display_component(pc, j):
    r,g,b = getNC(pc,j)
    return display_from_RGB(r,g,b)

And use these to see the first two PC's:

In [ ]:
display_component(pca2,0)

 It looks like the contrast difference between the presidential head and the surroundings is the main key to doing the classifying.  Let's see the second PC:

In [ ]:
display_component(pca2,1)

 The second PC seems to capture general darkness of the image.

## Your Turn

Do a 5 dimensional PCA, get the variance explanation, and display the components:

In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
display_from_RGB(*getMean(pca5))

## Using a Logistic Classifier with CV

Recall for CV we do the train/test split not once but multiple times on a grid of possible values for the optimal parameter `C` and for each `C` we:

1. create `n_folds` folds. Since the data size is approx. 90 here, and we have 5 folds, we roughly split the data into folds of 17-18 values each, randomly. 
2. We then train on 4 of these folds, test on the 5th
3. We average the results of all such combinations
4. We move on to the next value of `C`, and find the optimal value that minimizes mean square error.
5. We finally use that value to make the final fit.

We provide the following helper functions to do this. Notice the structure of the `GridSearchCV` estimator in `cv_optimize`.

In [ ]:
def fit_logistic(X_train, y_train, reg=0.0001, penalty="l2"):
    clf = LogisticRegression(C=reg, penalty=penalty)
    clf.fit(X_train, y_train)
    return clf

def cv_optimize(X_train, y_train, paramslist, penalty="l2", n_folds=10):
    clf = LogisticRegression(penalty=penalty)
    parameters = {"C": paramslist}
    gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(X_train, y_train)
    return gs.best_params_, gs.best_score_

def cv_and_fit(X_train, y_train, paramslist, penalty="l2", n_folds=5):
    bp, bs = cv_optimize(X_train, y_train, paramslist, penalty=penalty, n_folds=n_folds)
    print "BP,BS", bp, bs
    clf = fit_logistic(X_train, y_train, penalty=penalty, reg=bp['C'])
    return clf

We also provide a helper to show classification boundaries as well as distinguish test and training points:

In [ ]:
def points_plot(Xtr, Xte, ytr, yte, clf):
    X=np.concatenate((Xtr, Xte))
    h = .02
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 50),
                         np.linspace(y_min, y_max, 50))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    f,ax = plt.subplots()
    # Plot the training points
    ax.scatter(Xtr[:, 0], Xtr[:, 1], c=ytr, cmap=cm_bright)
    # and testing points
    ax.scatter(Xte[:, 0], Xte[:, 1], c=yte, cmap=cm_bright, marker="s", s=50, alpha=0.9)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=.4)
    cs2 = ax.contour(xx, yy, Z, cmap=cm, alpha=.4)
    plt.clabel(cs2, fmt = '%2.1f', colors = 'k', fontsize=14)
    return ax

Here is a way of doing a 70/30 train-test data split manually:

In [ ]:
is_train = np.random.uniform(0, 1, len(data)) <= 0.7
train_x, train_y = data[is_train], y[is_train]
test_x, test_y = data[is_train==False], y[is_train==False]

We *fit (find PC's) and transform* the training data, and then use the PC's to transform the test data:

In [ ]:
pca = PCA(n_components=2)
train_x = pca.fit_transform(train_x)
test_x = pca.transform(test_x)

We then do a cross-validated logistic regression and output the confusion matrix. Note the large value of the regularization factor. Why do you think this is the case?

In [ ]:
logreg = cv_and_fit(train_x, train_y, np.logspace(-4, 3, num=100))
pd.crosstab(test_y, logreg.predict(test_x), rownames=["Actual"], colnames=["Predicted"])

In [ ]:
logreg.coef_, logreg.intercept_

In [ ]:
points_plot(train_x, test_x, train_y, test_y, logreg)

Lets try a L1 penalty instead of L2. this is strictly not a correct thing to do since PCA and L2 regularization are both rotationally invariant. However, lets see what happen to the co-efficients:

In [ ]:
logreg_l1=cv_and_fit(train_x, train_y, np.logspace(-4, 3, num=100), penalty="l1")
pd.crosstab(test_y, logreg_l1.predict(test_x), rownames=["Actual"], colnames=["Predicted"])

In [ ]:
print logreg_l1.coef_, logreg_l1.intercept_

In [ ]:
points_plot(train_x, test_x, train_y, test_y, logreg_l1)

Notice L1 regularization supresses the intercept and reduces the importance of the second dimension. If one wants to minimize non-zero coefficients, one uses L1 regularization.

## Your Turn

Carry out a 5 dimensional PCA and then a logistic regression with both L2 and L1 penalties. Create crosstabs (confusion matracies) and print co-efficents for both. What do you find?

In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


# Lecture 8: Beyond Linearity

Linear models are relatively simple to describe and implement, and have advantages over other approaches in terms of interpretation and inference. However, standard linear regression can have significant limitations in terms of predictive power. This is because the linearity assumption is almost always an approximation, and sometimes a poor one.

We saw in the previous lectures improvements to the linear model by reducing the complexity of the model and hence it's variance but this will only get us up to a point.

We now relax the linearity assumption while still attempting to maintain as much interpretability as possible by looking at:

* Polynomial regression: extra predictors, obtained by raising each of the original predictors to a power
* Step functions: cut the range of a variable into $K$ distinct regions in order to produce a qualitative variable
* Regression splines: an extension of the above two methods
* Smoothing splines: use a smoothing penalty
* Local regression: similar to splines but use overlapping regions
* Generalized additive models: extend all these models to deal with many predictors

## Polynomial Regression

Let's use the `Wage` data and factorise the categorical data via dummy variables:

In [ ]:
wage_df = pd.read_csv("../data/Wage.csv",index_col ='id')
wage_df.head()

In [ ]:
wage_df["sex"] = pd.factorize(wage_df["sex"])[0]
wage_df["maritl"] = pd.factorize(wage_df["maritl"])[0]
wage_df["race"] = pd.factorize(wage_df["race"])[0]
wage_df["education"] = pd.factorize(wage_df["education"])[0]
wage_df["region"] = pd.factorize(wage_df["region"])[0]
wage_df["health"] = pd.factorize(wage_df["health"])[0]
wage_df["health_ins"] = pd.factorize(wage_df["health_ins"])[0]
wage_df.head()

Generally speaking, it is unusual to use a power greater than 3 or 4 for a feature and fit a model for it. For higher powers the polynomial curve can become overly flexible and can take on some very strange shapes, especially at the boundaries.

Let us think about fitting a degree-4 polynomial of `wage` as a function of `age`. Even though this is a linear regression model like any other, the individual coefficients are not of particular interest. Instead, we look at the entire fitted function.

## Orthogonal Polynomial Regression

We could of course construct new age features with increasing powers, but we have to keep in mind an issue that can pop-up in practice. Powers of say `age` could be correlated with one another and regression on correlated  predictors leads to unstable coefficients: the coefficients from an order-3 polynomial regression might change drastically when moving to an order-4 regression. Also if values in our feature column take large values then their powers will grow even larger leading to a poorly conditioned matrix when it comes to solving gradient descent. Larger values will also mean smaller coefficients leading to under-flow problems or coefficients which are hard to interpret.

We can fix this by using *orthogonal polynomial basis*. This is just a change in the coordinate system so has no effect on the regression.

The following helper `poly()` does just this:

In [ ]:
# from http://davmre.github.io/python/2013/12/15/orthogonal_poly/
def poly(x, degree = 1):
    n = degree + 1
    x = np.asarray(x).flatten()
    if(degree >= len(np.unique(x))):
        print("'degree' must be less than number of unique points")
        return
    xbar = np.mean(x)
    x = x - xbar
    X = np.fliplr(np.vander(x, n))
    q, r = np.linalg.qr(X)

    z = np.diag(np.diag(r))
    raw = np.dot(q, z)
    
    norm2 = np.sum(raw ** 2, axis=0)
    alpha = (np.sum((raw ** 2) * np.reshape(x,(-1,1)), axis=0) / norm2 + xbar)[:degree]
    Z = raw / np.sqrt(norm2)
    return Z, norm2, alpha

X = poly(wage_df["age"].values, 4)[0]
X[0:5, 1:]

In [ ]:
y = wage_df["wage"].values
reg = LinearRegression()
reg.fit(X, y)
print "Intercepts:", reg.intercept_
print "Coefficients:", reg.coef_
ax = wage_df.plot(x="age", y="wage", style="o")
ax.set_ylabel("wage")
# The poly() method cannot return features for a single X value, 
# so we have to plot the raw predictions.
age = wage_df["age"].values
ypred = reg.predict(X)
polyline = np.poly1d(np.polyfit(age, ypred, 4))
xs = range(int(np.min(age)), int(np.max(age)))
ys = polyline(xs) 
ax.plot(xs, ys, 'r', linewidth=2.5)

## Polynomial Logistic Regression

It seems like the wages are from two distinct populations: there appears to be a high earners group earning more than $250K per year as well as a low earners group. We can treat `wage` as a binary variable by splitting it into these two groups. Logistic regression can then be used to predict this binary response, using polynomial functions of `age` as predictors.

In [ ]:
X = np.vander(wage_df["age"],5)
y = wage_df["wage"].map(lambda w: 1 if w > 250 else 0).values
reg = LogisticRegression()
reg.fit(X, y)
print "Intercepts:", reg.intercept_
print "Coefficients:", reg.coef_

ypred = reg.predict(X)
print "MSE:", mean_squared_error(y, ypred)

xs = range(min(wage_df["age"]), max(wage_df["age"]))
plt.plot(xs, reg.predict_proba(np.vander(xs,5)))
plt.xlabel("age")
plt.ylabel("p(wage | age)")

## Splines

Now we discuss a flexible class of basis functions that extends upon the polynomial regression approach above.

Instead of fitting a high-degree polynomial over the entire range of X, piecewise polynomial regression involves fitting separate low-degree polynomials over different regions of $X$.

E.g. a piecewise cubic polynomial works by fitting:

$$
y_i = \beta_0 + \beta_1 x_1 + \beta_1 x_i + \beta_2 x_i^2 + \beta_3 x_i^3 + \epsilon_i,
$$

where the coefficients $\beta_0$, $\beta_1$, $\beta_2$, and $\beta_3$ differ in different parts of the range of $X$. The points where the coefficients change are called knots.

So a model with one knot at $c$ will need to fit two models one for $x_i < c$ and one for $x_i \ge c$. Using more knots leads to a more flexible piecewise polynomial. The problem is that the model will be discontinuous and looks ridiculous!

### Constraints

To fix this problem we can fit under a constraint: the fitted curve must be continuous. We do this by asking that the  first and second derivatives of the piecewise polynomials are continuous at the knots. These models are known as splines but unfortunately, splines can have high variance at the outer range of the predictors that is, when $X$ takes on either a very small or very large value.


### Choosing the Location of Knots

The regression spline is most flexible in regions that contain a lot of knots, because in those regions the polynomial coefficients can change rapidly. Hence, one option is to place more knots in places where we feel the function might vary most rapidly, and to place fewer knots where it seems more stable. 

While this option can work well, in practice it is common to place knots in a uniform fashion. Another way is to knot location at the 25th, 50th, and 75th percentiles of $X$. 

Another option is to use Mean shift clustering to discover “blobs” in a smooth density of samples.  It is a centroid-based algorithm, which works by updating candidates for centroids to be the mean of the points within a given region. These candidates are then filtered in a post-processing stage to eliminate near-duplicates to form the final set of centroids. 

In [ ]:
ges = wage_df["age"].values
wages = wage_df["wage"].values
X = np.vstack((ages, wages)).T

# cluster points to find the knot location
msc = MeanShift()
msc.fit(X)
knots = msc.cluster_centers_[:, 0]

# fit a spline over the points
spl = LSQUnivariateSpline(ages, wages, knots)
xs = range(np.min(ages), np.max(ages))
ys = spl(xs)

# plot the points and the spline
ax = wage_df.plot(x="age", y="wage", style="o")
ax.set_ylabel("wage")
ax.plot(xs, ys, 'r', linewidth=2.5)

### Choosing the Number of Knots

By now this should seem familiar:  use cross-validation. With this method, we remove a portion of the data, fit a spline with a certain number of knots to the remaining data, and then use the spline to make predictions for the held-out portion. We repeat this process multiple times until each observation has been left out once, and then compute the overall cross-validated RSS. This procedure can be repeated for different numbers of knots and we choose the number with the lowest RSS.

## Smoothing Splines

We now introduce a somewhat different approach that also produces a spline. Recall in fitting a smooth curve to a set of data, what we really want to do is find some function, $g(x)$ with small RSS = $\sum_{i=1}^{n} (y_i - g(x_i))^2$. But if we do not put a contraint on $g(x)$ then we can always make RSS = 0 by choosing a very flexible function that interpolates all of the $y_i$. Our model will overfit. What we really want is a function that makes RSS small but is also smooth.

A natural approach is to find the function $g$ that minimises:

$$
\sum_{i=1}^{n} (y_i - g(x_i))^2 + \lambda \int g''(t)^2\,\mathrm{d}t,
$$

where $\lambda$ is non-negative tuning parameter. The function that minimises this is called a smoothing spline. The second derivative is a measure of roughness and we are forcing the function to be smooth with this penalty. When $\lambda$ is huge we will get a straight line and when it is small the function will be very jumpy.

### Choosing $\lambda$

It turns out that we can compute the LOOCV error very efficiently for smoothing splines and thus we choose the $\lambda$ with the lowest LOOCV RSS error.

## Local Regression

Local regression is a different approach for fitting flexible non-linear functions, which involves computing the fit at a target point $x_0$ using only the nearby training observations.

Formally local regression at $X=x_0$ involves:

1. Gather the fraction $s=k/n$ of traning points whose $x_i$ are closest to $x_0$
2. Assign a weight $K_{i0}=K(x_i,x_0)$ to each point in this neighbourhood such that points further away from $x_0$ have zero weight and the closer ones have a heigher weight. All but the nearest $k$ get weight zero.
3. Fir a weighted least square regression of the $y_i$ on the $x_i$ using the above weights by finding the coefficients that minimise:
$$
\sum_{i-1}^{n}K_{i0}(y_i - \beta_0 -\beta_1 x_i)^2.
$$
4. The model at $x_0$ is given by $\hat{f}(x_0)=\hat{\beta}_0 + \hat{\beta}_1 x_0$.

This method however suffers from the curse of dimensionality and usually performs poorly for $p$ larger than 3 or 4.